In [223]:
import os

os.getcwd()
os.chdir(r'C:\Users\justi\Github\advent_of_code\day 9')

In [226]:
with open('input.txt') as f:
    lines = f.readlines()

lines = [x[:-1] for x in lines]
#print(lines)

In [227]:
import numpy as np

def split(string):
    return [item for item in string]

input = [split(x) for x in lines]
input = np.matrix(input)
input = input.astype(int)
input


matrix([[9, 2, 3, ..., 8, 9, 9],
        [8, 9, 6, ..., 9, 9, 8],
        [7, 8, 9, ..., 9, 8, 7],
        ...,
        [6, 4, 2, ..., 9, 8, 8],
        [6, 5, 3, ..., 4, 9, 9],
        [7, 6, 4, ..., 2, 3, 4]])

In [5]:
# i = row
# x = column

def find_nines(data):
    nines = np.array(np.where(data == 9))
    nines = np.stack((nines[0], nines[1]), axis = 1)  # reshape 2 arrays into 1 array of list of points in the matrix
    return nines

def make_tracker(data):
    nines = find_nines(data)
    size = data.shape
    tracker = np.zeros(size) # score card
    for n in nines:
        tracker[n[0], n[1]] = 9
    return tracker



def submatrix(data, i, x):  # submatric creator 
    collect = None
    if i == 0 and x == 0:                       # top left corner
        result = data[np.ix_([i,i+1],[x,x+1])]  # returns 2x2 matrix
        result[1,1] = 99  # not worried about diagonals
        if result.min() == result[0,0] and result.min() < 9:
            collect = [result.min(),(i,x)] # collect the value to keep
            
    elif i ==0 and x == input.shape[1] - 1:     # top right corner
        result = data[np.ix_([i,i+1], [x-1,x])]  # returns 2x2 matrix
        result[1, 0] = 99  # not worried about diagonals
        if result.min() == result[0,1] and result.min() < 9:
            collect = [result.min(),(i,x)] # collect the value to keep
            
    elif i== input.shape[0]-1 and x == 0:       # bottom left corner
        result = data[np.ix_([i-1,i], [x,x+1])] 
        result[0,1] = 99  # not worried about diagonals
        if result.min() == result[1,0] and result.min() < 9:
            collect = [result.min(),(i,x)] # collect the value to keep
            
    elif i == input.shape[0] - 1 and x == input.shape[1] - 1: # bottom right corner
        result = data[np.ix_([i-1,i], [x-1,x])]
        result[0,0] == 99
        if result.min() == result[1,1] and result.min() < 9:
            collect = [result.min(),(i,x)]
            
    elif i == 0:                                # top row
        result = data[np.ix_([i,i+1], [x-1, x, x+1])] # returns 3x2 matrix
        result[1,0] = 99
        result[1,2] = 99
        if result.min() == result[0,1] and result.min() < 9:
            collect = [result.min(),(i,x)]
            
    elif i == input.shape[0] - 1:               # bottom row
        result = data[np.ix_([i-1,i], [x-1, x, x+1])]
        result[0,0] = 99
        result[0,2] = 99
        if result.min() == result[1,1] and result.min() < 9:
            collect = [result.min(),(i,x)]
            
    elif x == 0:                                # left edge column
        result = data[np.ix_([i-1, i, i+1], [x,x+1])]
        result[0,1] = 99
        result[2,1] = 99
        if result.min() == result[1,0] and result.min() < 9:
            collect = [result.min(),(i,x)]
    
    elif x == input.shape[1] - 1:               # right edge column
        result = data[np.ix_([i-1, i, i+1], [x-1,x])]
        result[0,0] = 99
        result[2,0] = 99
        if result.min() == result[1,1] and result.min() < 9:
            collect = [result.min(),(i,x)]
    
    else:                                       # everything else that is not on an edge or in a corner
        result = data[np.ix_([i-1, i, i+1], [x-1, x, x+1])] # returns 3x3 matrix
        result[0,0] = 99
        result[2,0] = 99
        result[0,2] = 99
        result[2,2] = 99
        if result.min() == result[1,1] and result.min() < 9:
            collect = [result.min(),(i,x)]
    
    return collect

def add_collect(input):
    lowest_list = []
    
    for i in range(input.shape[0]):
        for x in range(input.shape[1]):
            lowest = submatrix(input, i,x)
            if lowest != None:
                lowest_list.append(lowest)
    
    low = [x[0] for x in lowest_list]  # the lowest number
    idx = np.array([x[1] for x in lowest_list])  # index of the lowest number in the overall matrix

    risk_level = [x+1 for x in low]    # calculate the risk level
    return sum(risk_level), low, idx





In [6]:



risk, low, low_idx = add_collect(input)


### <font color='green'>--- Part Two ---</font>

In [272]:
class Graph:
    
    def __init__(self, graph):
        self.row = graph.shape[0]
        self.col = graph.shape[1]
        self.graph = graph
        
    # function to determine if the number should even be evaluated in DFS
    def safety(self, i, j, visited):
        # row number is in range
        # col number is in range
        # have not visited this number
        if (i >= 0 and i < self.row and
            j >= 0 and j < self.col and
            visited[i][j] == 1):
            return visited[i][j]
        
    # function to do DFS for a 2D matrix (considering 8 neighbors only)
    def DFS(self, i, j, visited, count):
        
        # directions
        dirs = [(0, 1), (1, 0), (0, -1), (-1,0)]
        
        # mark cell as visited
        visited[i][j] = count
        
        # recursion for all neighboring numbers
        for k in range(4):
            if self.safety(i + dirs[k][0], j + dirs[k][1], visited):
                self.DFS(i + dirs[k][0], j + dirs[k][1], visited, count)
                      
    # main function that returns count of islands in the matrix
    def countIslands(self):
        nodes = []
        visited = np.zeros(self.graph.shape)   
        visited = np.where(self.graph == 9, visited, 1)
        # initialize count = 0 and traverse through all cells
        count = 0
        for i in range(self.row):
            for j in range(self.col):
                if visited[i][j] == 1 :
                    count -= 1
                    self.DFS(i,j,visited, count)
        # count the nodes in each island
        for c in range(abs(islands)):
            c += 1
            node_count = len(np.where(visited == -c)[0])
            nodes.append(node_count)
        # 3 largest basins
        nodes.sort()
        large_basins = np.prod(nodes[-3:])
        
        return -count, nodes, large_basins
        

In [274]:
# first test 2D matrix
basin = np.array([[2,1,9,9,9,4,3,2,1,0],
                  [3,9,8,7,8,9,4,9,2,1],
                  [9,8,5,6,7,8,9,8,9,2],
                  [8,7,6,7,8,9,6,7,8,9],
                  [9,8,9,9,9,6,5,6,7,8]])
g = Graph(basin)
islands,  nodes, large_basins = g.countIslands()
print(f'Islands found: {islands}')
print(f'Number of nodes in each island: {nodes}')
print(f'Dot produce of 3 largest basins: {large_basins}')

Islands found: 4
Number of nodes in each island: [3, 9, 9, 14]
Dot produce of 3 largest basins: 1134


In [276]:
g = Graph(input)
islands, nodes, large_basins = g.countIslands()
print(f'Islands found: {islands}')
print(f'Number of nodes in each island: {nodes}')
print(f'Dot produce of 3 largest basins: {large_basins}')

Islands found: 208
Number of nodes in each island: [2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 6, 6, 7, 7, 7, 7, 7, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12, 13, 13, 13, 13, 14, 14, 15, 15, 15, 15, 15, 17, 17, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 24, 24, 24, 25, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 28, 30, 31, 31, 32, 32, 32, 32, 32, 32, 33, 34, 34, 35, 35, 36, 36, 36, 36, 36, 37, 37, 37, 38, 38, 38, 38, 38, 39, 39, 40, 41, 41, 41, 44, 46, 46, 46, 46, 47, 48, 50, 51, 52, 53, 53, 56, 57, 58, 58, 59, 60, 60, 61, 62, 62, 63, 63, 66, 67, 67, 68, 72, 72, 73, 73, 75, 76, 76, 76, 78, 79, 79, 79, 81, 81, 81, 82, 87, 88, 88, 89, 89, 90, 91, 92, 92, 93, 93, 97, 99, 99, 101, 104, 106, 106, 107, 117]
Dot produce of 3 largest basins: 1327014
